In [118]:
import pandas as pd
import pprint
import glob

In [98]:
pd.options.display.max_colwidth = 200

In [79]:
from html.parser import HTMLParser

class CaseHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.data_parts = []

    def handle_data(self, data):
        self.data_parts.append(data)
        
def parse_text(case_text_html: str):
    parser = CaseHTMLParser()
    parser.feed(case_text_html)
    text = "".join(parser.data_parts)
    parser.close()
    return text

In [141]:
len(glob.glob("Дела/Сбыт/raw_csv/*"))

70

In [139]:
for fpath in glob.glob("Дела/Сбыт/raw_csv/*"):
    try:
        fname = fpath.split("/")[-1].split(".csv")[0]+".txt"
        df = pd.read_csv(fpath)
        text = parse_text(df[df["comment"]=="Текст документа"]['value'].values[0])
        with open("Дела/Сбыт/txt/"+fname, "w") as f:
            f.write(text)
    except:
        print(fname)

In [76]:
df = pd.read_csv("contraband_cases/Уголовное_дело_1-1_2018_(1-67_2017;)_2018-03-28T01:00:00.csv")

In [110]:
df[df["comment"]=="Уровнь суда"]["value"]

Series([], Name: value, dtype: object)

In [116]:
def get_defendants(defendants: pd.Series):
    names = []
    for i, text in defendants.iteritems():
        name = text
        names.append(name)
    return names

get_defendants(df[df["comment"]=="Лица"]["value"])

['Мельник А.В..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.228.1 ч.4 п.г; ст.33 ч.3-ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Диденко Ю.А..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.228.1 ч.4 п.г; ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Ангелов Г.В..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.228.1 ч.4 п.г; ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Саяшкина Е.А..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.33 ч.5-ст.228.1 ч.4 п.г; ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Саяшкина Е.А..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.33 ч.5-ст.228.1 ч.4 п.г; ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Ангелов Г.В..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.228.1 ч.4 п.г; ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Диденко Ю.А..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.228.1 ч.4 п.г; ст.229.1 ч.3 УК РФ.  ...  .ОБВИНИТЕЛЬНЫЙ приговор', 'Мельник А.В..  ...  .28.03.2018.  ...  .ст. 30 ч.3, ст.228.1 ч.4 п.г; ст.33 ч.3-ст.229.1 ч.3 УК РФ.  ... 

In [159]:
import traceback

dfs = []
for fpath in glob.glob("Дела/Сбыт/raw_csv/*"):
    df = pd.read_csv(fpath)
    try:
        try:
            judgement_date = df[df["comment"]=="Дата решения"]["dateValue"].iloc[0]
        except IndexError:
            judgement_date = None
            print(df[df["comment"]=="Дата решения"]["dateValue"])
        df_parsed = pd.DataFrame.from_dict(
            {
                "Текст": [parse_text(df[df["comment"]=="Текст документа"]['value'].values[0])],
                "Дата решения": [judgement_date],
                "Уровень по субъекту": [df[df["comment"]=="Уровень по субъекту"]["longValue"].iloc[0]],
                "Год регистрации дела": [df[df["comment"]=="Год регистрации дела"]["longValue"].iloc[0]],
                "Статья УК РФ": [df[df["comment"]=="Статья УК РФ"]["value"].iloc[0]],
                "Тип дела": [df[df["comment"]=="Тип дела"]["value"].iloc[0]],
                "Субъект РФ": [df[df["comment"]=="Субъект РФ"]["value"].iloc[0]],
                "Округ РФ": [df[df["comment"]=="Округ РФ"]["value"].iloc[0]],
                "Наименование суда": [df[df["comment"]=="Наименование суда"]["value"].iloc[0]],
                "Результат": [df[df["comment"]=="Результат"]["value"].iloc[0]],
                "Уровень суда": [df[df["comment"]=="Уровень суда"]["value"].iloc[0]],
                "Вид судопроизводства": [df[df["comment"]=="Вид судопроизводства"]["value"].iloc[0]],
                "Лица": [list(df[df["comment"]=="ФИО"]["value"])],
                "Подсудимые": [get_defendants(df[df["comment"]=="Лица"]["value"])],
                "Тип документа": [df[df["comment"]=="Тип документа"]["value"].iloc[0]],
                "Номер дела (материала)": [df[df["comment"]=="Номер дела (материала)"]["value"].iloc[0]],
                "Инстанция": [df[df["comment"]=="Инстанция"]["value"].iloc[0]],
                "Судья": [df[df["comment"]=="Судья"]["value"].iloc[0]],
                "Решение": [df[df["comment"]=="Решение"]["value"].iloc[0]]

            })
        dfs.append(df_parsed)
    except:
        print(fpath)
        print(traceback.format_exc())

Series([], Name: dateValue, dtype: object)
Series([], Name: dateValue, dtype: object)
Series([], Name: dateValue, dtype: object)


In [154]:
len(dfs)

70

In [161]:
result_df = pd.concat(dfs, ignore_index=True)

In [162]:
result_df.to_csv("Сводная_сбыт.csv")

In [164]:
result_df.head(1)

,Текст,Дата решения,Уровень по субъекту,Год регистрации дела,Статья УК РФ,Тип дела,Субъект РФ,Округ РФ,Наименование суда,Результат,Уровень суда,Вид судопроизводства,Лица,Подсудимые,Тип документа,Номер дела (материала),Инстанция,Судья,Решение
0,\r\n\r\n\r\n\r\n\r\nУголовное дело № 1-154/2018\r\nП Р И Г О В О Р\r\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\r\nгород Белгород 26 декабря 2018 года\r\nБелгородский районный суд Белгородской области в составе...,2018-12-26T01:00:00,2.0,2018.0,Статья 228 Часть 2; Статья 229.1 Часть 3,Производство в суде ПЕРВОЙ ИНСТАНЦИИ,Белгородская область,Центральный федеральный округ,Белгородский районный суд,Вынесен ПРИГОВОР,"Районный, городской, межрайонный суд",Уголовное дело,[nan],"[nan, nan]",Приговор,1-154/2018,Первая инстанция,Рощупкин Александр Федорович,Обвинительный приговор


In [ ]:
result_df.loc[0]["Текст"]

In [167]:
import ner

In [170]:
extractor = ner.EntityExtractor(filename="tmp", text=result_df.loc[0]["Текст"])

In [171]:
target_keys = ['Особый порядок', 'Судимость']
extractor.summary_dict

2019-07-21 19:32:13,358 CRITICAL link Could not parse link from filename tmp, 'NoneType' object has no attribute 'group', skipping
2019-07-21 19:32:13,498 WARNING drugs No drugs found in whole text, check matches: []
2019-07-21 19:32:13,587 WARNING drugs No drugs found in whole text, check matches: []


{'Ссылка': 'нет данных', 'Файл': 'tmp', 'Суд': 'белгородский районный суд белгородской области', 'Дата приговора': '2018/12/26', 'ФИО': 'бахтина в.а. и его защитника – адвоката сазончик н.н.', 'Особый порядок': 'нет', 'Судимость': 'да', 'Вид наказания': 'Лишение свободы', 'Срок наказания в месяцах': 'нет', 'Отбывал ли ранее лишение свободы': 'да', 'Наркотики': '', 'Главный наркотик': '', 'Размер': 'Крупный', 'Смягчающие обстоятельства': 'активное способствование раскрытию и расследованию преступлений, выразившееся в даче последовательных признательных показаний (п. «и» ч.1 ст. 61 ук рф), состояние его (информация скрыта) (ч.2 ст. 61 ук рф)', 'Отягчающие обстоятельства': 'рецидив преступлений'}